# 🧠 SynthMed Development Notebook
This Jupyter Notebook helps you develop, test, and package a **Watsonx Orchestrate Action** using the **Action Development Kit (ADK)**.

Follow the steps below to set up and test your custom action locally.

## 1. Environment Setup
Install dependencies and verify your environment.

In [ ]:
!pip install ibm-watsonx-orchestrate pyyaml

## 2. Import Required Libraries and Define Orchestrate Function
Load the necessary Python packages for ADK and YAML handling.

In [ ]:
import requests
import subprocess
import yaml
from google.colab import userdata

def run_orchestrate_command(command_args):
  """
  Executes an 'orchestrate' CLI command and returns the output.

  Args:
      command_args (list): A list of strings representing the 'orchestrate'
                           command and its arguments. For example:
                           ['tools', 'import', '-k', 'python', '-f', 'my_tool.py']

  Returns:
      tuple: A tuple containing (stdout, stderr, return_code).
  """

  try:
    # Prepend 'orchestrate' to the command arguments
    full_command = ['orchestrate'] + command_args

    # Execute the command
    process = subprocess.run(
        full_command,
        capture_output=True,
        text=True,  # Decode stdout and stderr as text
        check=True   # Raise CalledProcessError for non-zero exit codes
    )
    return process.stdout, process.stderr, process.returncode
  except subprocess.CalledProcessError as e:
    print(f"Error executing command: {e}")
    print(f"Stdout: {e.stdout}")
    print(f"Stderr: {e.stderr}")
    return e.stdout, e.stderr, e.returncode
  except FileNotFoundError:
    print("Error: 'orchestrate' command not found. Ensure the watsonx Orchestrate ADK is installed and in your PATH.")
    return "", "Error: 'orchestrate' command not found.", 1

## 3. Create and Activate SynthMed Environment
Set up the Orchestrate environment using user-defined API_URL and API_KEY.

In [ ]:
apiurl = userdata.get("API_URL")
apikey = userdata.get("API_KEY")
apicrn = userdata.get("API_CRN")

# Remove existing environment.
#stdout, stderr, return_code = run_orchestrate_command(['env', 'remove', '-n', 'synthmed'])
#if return_code == 0:
#  print("Environment synthmed removed successfully.")
#else:
#  print("Environment synthmed did not exist.")

# Add new environment.
stdout, stderr, return_code = run_orchestrate_command(
  ['env', 'add', '-n', 'synthmed', '-u', apiurl, '--type', 'ibm_iam'])
if return_code == 0:
  print("Environment synthmed added successfully.")
else:
  print("Environment synthmed not added or already existing.")

# Activate new environment.
stdout, stderr, return_code = run_orchestrate_command(['env', 'activate', '-a', apikey, 'synthmed'])
if return_code == 0:
  print("Environment synthmed activated successfully.")
else:
  print("Environment synthmed not activated successfully")

## 4. Execute Orchestrate Commands
Test webchat, agent import, kb import, tool import.


In [ ]:
agent_name = "synthmed_agent"
environment = "live"

agent_main_yaml = 'synthmed_autism_agent.yaml'
agent_autism_yaml = 'synthmed_autism_agent.yaml'

# Import main agent.
stdout, stderr, return_code = run_orchestrate_command(
    ['agents', 'import', '--file', agent_main_yaml])
if return_code == 0:
  print(agent_main_yaml + " successfully imported.")
else:
  print(agent_main_yaml + " not imported.")

# Import autism agent.
stdout, stderr, return_code = run_orchestrate_command(
    ['agents', 'import', '--file', agent_autism_yaml])
if return_code == 0:
  print(agent_autism_yaml + " successfully imported.")
else:
  print(agent_autism_yaml + " not imported.")

# Create channel.
stdout, stderr, return_code = run_orchestrate_command(
    ['channels', 'webchat', 'embed', '--agent-name', agent_name, '--env', environment])
if return_code == 0:
  print("Channel to synthmed_agent created.")
else:
  print("Channel to synthmed_agent not created.")

## 5. Run SynthMed as a web app.
Use the ActionRunner to run your action as Orchestrate would.

In [ ]:
runner = ActionRunner(action_path=action_path)
runner.run()

## 6. Package the Action for Upload
Once validated, package your action into a `.zip` for uploading to Watsonx Orchestrate.

In [ ]:
!adk package $action_path

✅ **Next Steps:**
- Log in to the [Watsonx Orchestrate Developer Portal](https://watsonx.ibm.com/orchestrate)
- Upload the generated `.zip` package
- Test it in the sandbox or production workspace